In [9]:
import sys
sys.path.append('/Users/ahalev/Dropbox/Avishai/gradSchool/internships/totalInternship/pymgrid_git')
path = sys.path[len(sys.path)-1]
import pymgrid.MicrogridGenerator as mg
import utils.DataGenerator as dg
import algos.Control as control
import pickle
import importlib

def reload():
    importlib.reload(dg)
    importlib.reload(mg)
    importlib.reload(control)
reload()  

In [5]:
path = sys.path[len(sys.path)-1]
env = mg.MicrogridGenerator(path = path,nb_microgrid=2)
env = env.load('pymgrid25')

In [ ]:
comparison_list = []
forecast_accuracies = [50, 70, 85]
percentiles = [0.25, 0.5, 0.75]

sampling_args = dict()
sampling_args['push_individual_vals'] = True # For pv data
sampling_args['push_individual_ratio'] = 0.5 # For pv data

total_iter = len(env.microgrids)*len(forecast_accuracies)*len(percentiles)


for j, microgrid in enumerate(env.microgrids):
    for k, accuracy in enumerate(forecast_accuracies):
        for l, percentile in enumerate(percentiles):
            
            m = (j*len(forecast_accuracies)+k)*len(percentiles)+l
            print(round(100*m/total_iter,2),'%')
        
            SAA = control.SampleAverageApproximation(microgrid, preset_to_use=accuracy)
            mape = SAA.validate_forecasts(forecasts = SAA.forecasts, aggregate = True)
            
            mpc_output = SAA.run_deterministic_on_forecast()
            saa_output = SAA.run(optimal_percentile = percentile, use_previous_samples=False, **sampling_args)
            
            d = dict(microgrid=microgrid, percentile=percentile, accuracy=accuracy, mpc_output=mpc_output, saa_output = saa_output)
            comparison_list.append(d)


0.0 %
Using preset forecast arguments


/Users/ahalev/Dropbox/Avishai/gradSchool/internships/totalInternship/pymgrid_git/algos/Control.py:191: RuntimeWarning:

invalid value encountered in true_divide



Creating sample 0
Creating sample 1
Creating sample 2
Creating sample 3
Creating sample 4
Creating sample 5
Creating sample 6
Creating sample 7
Creating sample 8
Creating sample 9


In [ ]:
filename = 'comparison_algos'
outfile = open(filename,'wb')
pickle.dump(comparison_list,outfile)
outfile.close()